# Target-Specific De Novo Peptide Binder Design with DiffPepBuilder

This notebook demonstrates how to use the DiffPepBuilder package to design peptides that bind to a target protein. We provide an example of the target ALK1 (Activin Receptor-like Kinase 1, PDB ID: [6SF1](https://www.rcsb.org/structure/6SF1)) to demonstrate the procedures of generating peptide binders.